[![Colab Badge](https://img.shields.io/badge/Open_in_Colab-blue?style=for-the-badge)][colab-link]
<a href="javascript:void(0);" onclick="openJupyterWidget('https://github.com/nmfs-opensci/nmfshackdays-2025/blob/main/topics-2025/2025-opendap/5-usgs.ipynb');">
    <img src="https://img.shields.io/badge/Open_in_JupyterHub-orange?style=for-the-badge" alt="JupyterHub Badge">
</a> [![Download Badge](https://img.shields.io/badge/Download-grey?style=for-the-badge)][download-link]

[download-link]: https://nmfs-opensci.github.io/NMFSHackDays-2025/topics-2025/2025-opendap/5-usgs.ipynb
[colab-link]: https://colab.research.google.com/github/nmfs-opensci/nmfshackdays-2025/blob/main/topics-2025/2025-opendap/5-usgs.ipynb
[jupyter-link]: https://nmfs-openscapes.2i2c.cloud/hub/user-redirect/lab?fromURL=https://raw.githubusercontent.com/nmfs-opensci/nmfshackdays-2025/main/topics-2025/2025-opendap/5-usgs.ipynb

## Overview

The USGS LPDAAC server requires NASA Earthdata login authentication, but doesn't require a EULA (as far as I know). However they have similar redirect issues as data that does require a EULA. I could not get the data into an xarray.

### Prerequisites

I assume you have a `.netrc` file at `~` (home). `~/.netrc` should look just like this with your username and password.

```
machine urs.earthdata.nasa.gov
        login yourusername
        password yourpassword
```

### Packages


In [22]:
import xarray as xr
import pydap.client

## USGS LPDAAC

Their OPeNDAP server also uses NASA Earthdata login authentication.

In [4]:
# load username and password
import netrc
# Get credentials from .netrc
auth_host = "urs.earthdata.nasa.gov"
try:
    credentials = netrc.netrc().authenticators(auth_host)
    if credentials:
        username, _, password = credentials
    else:
        raise ValueError("No credentials found in .netrc!")
except FileNotFoundError:
    raise FileNotFoundError("Could not find ~/.netrc. Ensure it exists and is configured correctly.")

In [33]:
url="https://opendap.cr.usgs.gov/opendap/hyrax/MOD13Q1.061/h09v06.ncml"

In [28]:
import pydap
from pydap.client import open_url
from pydap.cas.urs import setup_session
session = setup_session(username, password)

In [39]:
# this works
pydap_ds = open_url(url, session=session, protocol="dap4")

This doesn't work
```
store = xr.backends.PydapDataStore(pydap_ds)
ds = xr.open_dataset(store)
```

This doesn't either. Gives the same redirect error.
```
url="dap4://opendap.cr.usgs.gov/opendap/hyrax/MOD13Q1.061/h09v06.ncml"
ds = xr.open_dataset(url, engine="pydap", session=session)
```

In [40]:
# this is another url that works with pydap
url = "https://opendap.cr.usgs.gov/opendap/hyrax/ECOSTRESS/ECO2CLD.001/2018.07.09/ECOSTRESS_L2_CLOUD_00048_001_20180709T204901_0601_03.h5"
pydap_ds = open_url(url, session=session, protocol="dap4")

## Conclusion

Working with USGS data on its OPeNDAP server is hard if you want an xarray object.

## References

* <https://github.com/pydap/pydap/issues/188>
* <https://nsidc.org/data/user-resources/help-center/how-do-i-access-data-using-opendap#anchor-using-a-command-line-interface>

